# Rozdzial 6.3


### *6.3 oraz 6.4
### Ogólne całą metodyka opiera się na analizie statystycznej Załączony najlepszy schemat implementacji deszyfracij szyfru ADFGXV 
### Plan działania:
1. Kto go zastosował oraz kto go złamał
2. Wyjaśnienie implementacij algorytmu deszyfracji
### 1. Zastosowanie oraz złamanie
Szyfr jest oczywiście Niemiecki z okresu I Wojny Światowej złamał go Francuski kryptolog porucznik Georges Painvina. Na początku Painvin zdołał jedynie ustalić, że szyfrowanie opiera się na szachownicy Polibiusza, a do szyfrowania używane są klucze dzienne. Okazało się, że niektóre z wiadomości nadawanych tego samego dnia miała identyczne początki. Jednocześnie wyodrębnił litery służące do szyfrowania (ADFGVX), a następnie sprawdził jak często występują poszczególne pary liter. Najczęstszą występującą kombinacją było DG – wywnioskował stąd, że jest to litera e, która w języku niemieckim występuje najczęściej. Na podstawie dalszej analizy statystycznej udało mu się zrekonstruować całą tablicę. Dzięki temu był w stanie łamać klucze dzienne niemieckich szyfrantów. Od 1 czerwca przechwytywane szyfrogramy zawierały dodatkową literę – V. Oznaczało to, że Niemcy zmienili szyfr – była to nowa wersja, ADFGVX. Ponownie, wykorzystując analizę statystyczną, Painvin odtworzył szachownicę służącą do szyfrowania.
### 2.Algorytm deszyfrujący przedstawiający implementację 
Algorytm wspinaczkowy 
![](zdjecia/algorytm_łamania_szyfrogramu.png)/
Każda runda rozpoczynamy  od fazy wstępnej, BestRandomTr(10000), która wybiera najlepszy klucz od 10 000 losowych klawiszy.
W fazie wstępnej, a do wspinaczki, algorytm wykorzystuje specjalistyczną metodę punktacji ICpairs, który jest oparty na Indeksie Zbieżności (patrz punkt 3. 2. 3"Musze się na tym skupić przczytać bo nie wiem o co dokładnie chodzi"), ale w przeciwieństwie do standardowego IC, nie jest obliczany na podstawie monogramów. Zamiast tego jest on obliczany  dla każdego kandydującego klucza transpozycji CandidateTr, stosujemy odwrotność tego klucza transpozycji („anuluj transpozycję”) do kod C, czyli TrInv(CandidateTr,C). Następnie dzielimy wynikową sekwencję symboli ADFGVX na pary (sąsiadujących symboli) i obliczamy ich indeks zbieżności, który jest, ICpairs(). Należy pamiętać, że jeśli klucz kandydata jest równy poprawnemu oryginalnemu kluczowi szyfrowania (CandidateTr = Ktr), każda para w sekwencji po cofnięciu transpozycji reprezentuje pojedynczy oryginalny symbol tekstowy (A-Z, 0–9) po podstawieniu. Jako (monogram) IC zwykłego tekstu P zaszyfrowane z podstawieniem monoalfabetycznym jest zawsze równe IC zwykłego tekstu, w przypadku
poprawnego klucza transpozycji kandydata (CandidateTr = Ktr)
Relacja ta jest kluczem do realizacji tego ataku „dziel i rządź”, wykorzystując
ICpairs, który pozwala na odzyskanie klucza transpozycji z pominięciem podstawienia klucz (Brutal Force). Podczas szyfrowania faza substytucji ADFGVX mapuje symbole zwykłego tekstu na pary symboli ADFGVX. Faza transpozycji dzieli te pary symboli, tworząc więcej przypadkowych kombinacji. Im bardziej algorytm jest w stanie zmienić położenie elementów tych oryginalnych par, używając częściowo poprawnego klucza transpozycji kandydata, tym wyższa wartość ICpairs powinna być. Najwyższa wartość ICpairs jest oczekiwana, gdy klucz transpozycji kandydata jest poprawny. Zaimplementowaliśmy zestaw transformacji używanych przy każdej iteracji i zastosowaliśmy do bieżącego  klucza transpozycji   (NeighborTrKeys(BestTr)).Transformacje te składają się z:
1. Proste zamiany dowolnych dwóch elementów w kluczu transpozycji.
2. Swapy dowolnych dwóch segmentów (lub różnych długościach) kolejnych elementów w klucza.
3. Obracanie (cyklicznie) segmentu czyli kolejnych elementów w kluczu (w dowolnej pozycji, i różnej długości).
4. Odwrócenie klucza transpozycji, tak aby ostatni element stał się pierwszym i tak dalej.
5. Zamiana elementów każdej pary w kluczu. W przypadku klucza o nieparzystej długości, pomińmy ostatni element klucza.

Pierwsze trzy typy – pojedyncze swapy i transformacje segmentowe, nie mają charakteru zakłócającego. Co do w przypadku szyfru transpozycji kolumnowej (sekcja 5. 3. 2)("Krystian tu dopisze"), transformacje segmentów są obowiązkowe, a nasz algorytm nie powiedzie się, jeśli je będziemy usuwać. Dwa ostatnie typy były empirycznie odkryte. Zanim je  wprowadziliśmy wspinaczka często utknęła na literach z wysokim wynikiem, które były podobne do poprawnego klucza, ale z kluczowymi elementami odwróconymi, lub z każdymi dwoma kolejnymi elementami odwróconymi.Algorytm ten sprawdza się dobrze dla kluczy transpozycji o nieparzystej długości. W przypadku kluczy transpozycji o parzystej długości często się nie powiedzie. Aby zilustrować problem, rozważamy następujący klucz transpozycji z 8 elementami (równej długość): Ktr = (4,6,2,3,7,5,1,8), a niektóre klucze zastępcze, Ksub. Szyfrujemy zwykły tekst P, najpierw nakładamy Ksub i otrzymujemy sekwencję symboli ADFGVX, gdzie każda para reprezentuje oryginalny symbol tekstowy. Następnie aplikujemy Ktr i aby uzyskać kod C. Zastosujemy algorytm wspinaczki do tego szyfru adn zakładamy, że znamy długość klucza transpozycji, |K| = 8. Na pewnym etapie wspinaczki, możemy uzyskać następujący klucz kandydata: CandidateTr = (7,5,4,6,1,8,2,3). Zauważmy, że w  kluczu kandydujący, oryginalne pary kluczowych elementów, np. (4, 6) lub (2, 3), zostały poprawnie
zrekonstruowane, ale względna kolejność par jest błędna. Ze względu na charakter transpozycji kolumnowej, jeśli zastosujemy odwrotność CandidateTr, czyli TrInv(CandidateTr,C), otrzymamy sekwencję symboli ADFGVX, w której dwa elementy każdej z oryginalnych par ADFGVX (po podstawieniu, ale przed transpozycją), są teraz obok siebie. W rezultacie ICpairs(TrInv(CandidateTr,C)) = IC(P), chociaż KandydatTr!(różni się od Ktr) = Ktr. Innymi słowy, mamy fałszywie wysokie ICpairs. W wyniku tego zjawiska nie możemy polegać tylko na ICpairs jako funkcji scoringowej, aby odzyskać klucz transpozycji o równej długości. Gdy długość klucza jest nieparzysta, zjawisko to nie występuje. Aby poradzić sobie z literami transpozycji o równej długości, dostosowaliśmy funkcję scoringową, tak aby łączny poprzedni pomiar (ICpairs) z IC czworokątów (ICquads). Dzielimy na sekwencje TrInv(CandidateTr,C)(otrzymamy po cofnięciu transpozycji) do czworokątów dla Symboli ADFGVX obliczyć ich IC. Zauważ, że jeśli klucz kandydata jest poprawny, to jest CandidateTr = Ktr, każda czwórka symboli ADFGVX (po cofnięciu transpozycji) reprezentuje zatem bigram symboli tekstowych (przed zastąpieniem). Po zmodyfikowaniu algorytmu za pomocą tej ulepszonej metody punktacji, algorytm wykonał również dobrze transpozycję otrzymaliśmy klucze o równej długości. 
Aby ocenić wydajność algorytmu, przeprowadziliśmy następujący eksperyment. Podzieląc kryptogramy na grupy według kluczy, którymi były zaszyfrowane, uruchomiliśmy algorytm na pierwszych n kryptogramów każdej grupy, zaczynając od n = 10, i redukując n za każym razem, aż algorytm nie odzyska klucza. Zamiast wybierać najdłuższy lub najczystsze kryptogramy z każdej grupy, właśnie wybraliśmy pierwsze n kryptogramów z każdej grupy.Dlatego n kryptogramów wybranych z każdej grupy (klucza) może się różnić pod względem długości lub liczba błędów transkrypcji. W tabeli 6. 1 pokazujemy minimalną liczbę n kryptogramów wymagane przez algorytm, aby umożliwić pełne odzyskanie klucza. Widać, że na 6 klawiszy z 14, tylko 2 kryptogramy o łącznej liczbie 388 do 566 symboli ADFGVX były wystarczające do odzyskania klucze transpozycji o długości od 16 do 23. Najgorszy przypadek był z kluczem długości 18 co wymagało 5 komunikatów o łącznej liczbie 1000 symboli ADFGVX. 